# Solving Linear Systems

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
%matplotlib inline

## Linear Systems

A [linear system of equations](https://en.wikipedia.org/wiki/System_of_linear_equations) is a collection of linear equations

\begin{align}
a_{0,0}x_0 + a_{0,1}x_2 + \cdots + a_{0,n}x_n & = b_0 \\\
a_{1,0}x_0 + a_{1,1}x_2 + \cdots + a_{1,n}x_n & = b_1 \\\
& \vdots \\\
a_{m,0}x_0 + a_{m,1}x_2 + \cdots + a_{m,n}x_n & = b_m \\\
\end{align}

In matrix notation, a linear system is $A \mathbf{x}= \mathbf{b}$ where

$$
A = \begin{bmatrix}
a_{0,0} & a_{0,1} & \cdots & a_{0,n} \\\
a_{1,0} & a_{1,1} & \cdots & a_{1,n} \\\
\vdots & & & \vdots \\\
a_{m,0} & a_{m,1} & \cdots & a_{m,n} \\\
\end{bmatrix}
 \ \ , \ \
\mathbf{x} = \begin{bmatrix}
x_0 \\\ x_1 \\\ \vdots \\\ x_n
\end{bmatrix}
 \ \ , \ \
\mathbf{b} = \begin{bmatrix}
b_0 \\\ b_1 \\\ \vdots \\\ b_m
\end{bmatrix} 
$$

## Gaussian elimination

The general procedure to solve a linear system of equation is called [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination). The idea is to perform elementary row operations to reduce the system to its row echelon form and then solve.

### Elementary Row Operations

[Elementary row operations](https://en.wikipedia.org/wiki/Elementary_matrix#Elementary_row_operations) include:

1. Add $k$ times row $j$ to row $i$.
2. Multiply row $i$ by scalar $k$.
3. Switch rows $i$ and $j$.

Each of the elementary row operations is the result of matrix multiplication by an elementary matrix (on the left).
To add $k$ times row $i$ to row $j$ in a matrix $A$, we multiply $A$ by the matrix $E$ where $E$ is equal to the identity matrix except the $i,j$ entry is $E_{i,j} = k$. For example, if $A$ is 3 by 3 and we want to add 3 times row 2 to row 0 (using 0 indexing) then

$$
E_1 = \begin{bmatrix}
1 & 0 & 3 \\\
0 & 1 & 0 \\\
0 & 0 & 1
\end{bmatrix}
$$

Let's verify the calculation:

In [2]:
A = np.array([[1,1,2],[-1,3,1],[0,5,2]])
print(A)

[[ 1  1  2]
 [-1  3  1]
 [ 0  5  2]]


In [3]:
E1 = np.array([[1,0,3],[0,1,0],[0,0,1]])
print(E1)

[[1 0 3]
 [0 1 0]
 [0 0 1]]


In [4]:
E1 @ A

array([[ 1, 16,  8],
       [-1,  3,  1],
       [ 0,  5,  2]])

To multiply $k$ times row $i$ in a matrix $A$, we multiply $A$ by the matrix $E$ where $E$ is equal to the identity matrix except the $,i,j$ entry is $E_{i,i} = k$. For example, if $A$ is 3 by 3 and we want to multiply row 1 by -2 then

$$
E_2 = \begin{bmatrix}
1 & 0 & 0 \\\
0 & -2 & 0 \\\
0 & 0 & 1
\end{bmatrix}
$$

Let's verify the calculation:

In [5]:
E2 = np.array([[1,0,0],[0,-2,0],[0,0,1]])
print(E2)

[[ 1  0  0]
 [ 0 -2  0]
 [ 0  0  1]]


In [6]:
E2 @ A

array([[ 1,  1,  2],
       [ 2, -6, -2],
       [ 0,  5,  2]])

Finally, to switch row $i$ and row $j$ in a matrix $A$, we multiply $A$ by the matrix $E$ where $E$ is equal to the identity matrix except $E_{i,i} = 0$, $E_{j,j} = 0$, $E_{i,j} = 1$ and $E_{j,i} = 1$. For example, if $A$ is 3 by 3 and we want to switch row 1 and row 2 then

$$
E^3 = \begin{bmatrix}
1 & 0 & 0 \\\
0 & 0 & 1 \\\
0 & 1 & 0
\end{bmatrix}
$$

Let's verify the calculation:

In [7]:
E3 = np.array([[1,0,0],[0,0,1],[0,1,0]])
print(E3)

[[1 0 0]
 [0 0 1]
 [0 1 0]]


In [8]:
E3 @ A

array([[ 1,  1,  2],
       [ 0,  5,  2],
       [-1,  3,  1]])

### Implementation

Let's write function to implement the elementary row operations. First of all, let's write a function called `add_rows` which takes input parameters $A$, $k$, $i$ and $j$ and returns the NumPy array resulting from adding $k$ times row $j$ to row $i$ in the matrix $A$. If $i=j$, then let's say that the function scales row $i$ by $k+1$ since this would be the result of $k$ times row $i$ added to row $i$.

In [35]:
def add_row(A,k,i,j):
    "Add k times row j to row i in matrix A."
    n = A.shape[0]
    E = np.eye(n)
    if i == j:
        E[i,i] = k + 1
    else:
        E[i,j] = k
    return E @ A

Let's test our function:

In [10]:
M = np.array([[1,1],[3,2]])
print(M)

[[1 1]
 [3 2]]


In [11]:
add_row(M,2,0,1)

array([[7., 5.],
       [3., 2.]])

In [12]:
add_row(M,3,1,1)

array([[ 1.,  1.],
       [12.,  8.]])

Let's write a function called `scale_row` which takes 3 input parameters $A$, $k$, and $i$ and returns the matrix that results from $k$ times row $i$ in the matrix $A$.

In [34]:
def scale_row(A,k,i):
    "Multiply row i by k in matrix A."
    n = A.shape[0]
    E = np.eye(n)
    E[i,i] = k
    return E @ A

In [14]:
M = np.array([[3,1],[-2,7]])
print(M)

[[ 3  1]
 [-2  7]]


In [15]:
scale_row(M,3,1)

array([[ 3.,  1.],
       [-6., 21.]])

In [16]:
A = np.array([[1,1,1],[1,-1,0]])
print(A)

[[ 1  1  1]
 [ 1 -1  0]]


In [17]:
scale_row(A,5,1)

array([[ 1.,  1.,  1.],
       [ 5., -5.,  0.]])

Let's write a function called `switch_rows` which takes 3 input parameters $A$, $i$ and $j$ and returns the matrix that results from switching rows $i$ and $j$ in the matrix $A$.

In [33]:
def switch_rows(A,i,j):
    "Switch rows i and j in matrix A."
    n = A.shape[0]
    E = np.eye(n)
    E[i,i] = 0
    E[j,j] = 0
    E[i,j] = 1
    E[j,i] = 1
    return E @ A

In [19]:
A = np.array([[1,1,1],[1,-1,0]])
print(A)

[[ 1  1  1]
 [ 1 -1  0]]


In [20]:
switch_rows(A,0,1)

array([[ 1., -1.,  0.],
       [ 1.,  1.,  1.]])

## Examples

### Find the Inverse

Let's apply our functions to the augmented matrix $[M \ | \ I]$ to find the inverse of the matrix $M$:

In [21]:
M = np.array([[5,4,2],[-1,2,1],[1,1,1]])
print(M)

[[ 5  4  2]
 [-1  2  1]
 [ 1  1  1]]


In [22]:
A = np.hstack([M,np.eye(3)])
print(A)

[[ 5.  4.  2.  1.  0.  0.]
 [-1.  2.  1.  0.  1.  0.]
 [ 1.  1.  1.  0.  0.  1.]]


In [23]:
A1 = switch_rows(A,0,2)
print(A1)

[[ 1.  1.  1.  0.  0.  1.]
 [-1.  2.  1.  0.  1.  0.]
 [ 5.  4.  2.  1.  0.  0.]]


In [24]:
A2 = add_row(A1,1,1,0)
print(A2)

[[1. 1. 1. 0. 0. 1.]
 [0. 3. 2. 0. 1. 1.]
 [5. 4. 2. 1. 0. 0.]]


In [25]:
A3 = add_row(A2,-5,2,0)
print(A3)

[[ 1.  1.  1.  0.  0.  1.]
 [ 0.  3.  2.  0.  1.  1.]
 [ 0. -1. -3.  1.  0. -5.]]


In [26]:
A4 = switch_rows(A3,1,2)
print(A4)

[[ 1.  1.  1.  0.  0.  1.]
 [ 0. -1. -3.  1.  0. -5.]
 [ 0.  3.  2.  0.  1.  1.]]


In [27]:
A5 = scale_row(A4,-1,1)
print(A5)

[[ 1.  1.  1.  0.  0.  1.]
 [ 0.  1.  3. -1.  0.  5.]
 [ 0.  3.  2.  0.  1.  1.]]


In [28]:
A6 = add_row(A5,-3,2,1)
print(A6)

[[  1.   1.   1.   0.   0.   1.]
 [  0.   1.   3.  -1.   0.   5.]
 [  0.   0.  -7.   3.   1. -14.]]


In [29]:
A7 = scale_row(A6,-1/7,2)
print(A7)

[[ 1.          1.          1.          0.          0.          1.        ]
 [ 0.          1.          3.         -1.          0.          5.        ]
 [ 0.          0.          1.         -0.42857143 -0.14285714  2.        ]]


In [30]:
A8 = add_row(A7,-3,1,2)
print(A8)

[[ 1.          1.          1.          0.          0.          1.        ]
 [ 0.          1.          0.          0.28571429  0.42857143 -1.        ]
 [ 0.          0.          1.         -0.42857143 -0.14285714  2.        ]]


In [31]:
A9 = add_row(A8,-1,0,2)
print(A9)

[[ 1.          1.          0.          0.42857143  0.14285714 -1.        ]
 [ 0.          1.          0.          0.28571429  0.42857143 -1.        ]
 [ 0.          0.          1.         -0.42857143 -0.14285714  2.        ]]


In [32]:
A10 = add_row(A9,-1,0,1)
print(A10)

[[ 1.          0.          0.          0.14285714 -0.28571429  0.        ]
 [ 0.          1.          0.          0.28571429  0.42857143 -1.        ]
 [ 0.          0.          1.         -0.42857143 -0.14285714  2.        ]]


Let's verify that we found the inverse $M^{-1}$ correctly:

In [33]:
Minv = A10[:,3:]
print(Minv)

[[ 0.14285714 -0.28571429  0.        ]
 [ 0.28571429  0.42857143 -1.        ]
 [-0.42857143 -0.14285714  2.        ]]


In [34]:
result = Minv @ M
print(result)

[[ 1.00000000e+00  4.44089210e-16  2.22044605e-16]
 [-6.66133815e-16  1.00000000e+00 -2.22044605e-16]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


Success! We can see the result more clearly if we round to 15 decimal places:

In [35]:
np.round(result,15)

array([[ 1.e+00,  0.e+00,  0.e+00],
       [-1.e-15,  1.e+00, -0.e+00],
       [ 0.e+00,  0.e+00,  1.e+00]])

### Solve a System

Let's use our functions to perform Gaussian elimination and solve a linear system of equations $A \mathbf{x} = \mathbf{b}$.

Recall the linear system from a previous exercise: 

$$
\begin{array}{ r @{{}={}} r  >{{}}c<{{}} r  >{{}}c<{{}}  r }
x_1 &-& 3x_2 &+& 4x_3 &=&  1 \\
-2x_1 &+&  5x_2 &-& 7x_3 &=& 1 \\
x_1 &-& 5x_2 &+& 8x_3 &=& 5 \\
\end{array}
$$

Back then we used SymPy to solve it

In [2]:
import sympy as sy

In [69]:
A = sy.Matrix([[1,-3,4],[-2,5,-7],[1,-5,8]])
A

Matrix([
[ 1, -3,  4],
[-2,  5, -7],
[ 1, -5,  8]])

In [70]:
b = sy.Matrix([[1],[1],[5]])
b

Matrix([
[1],
[1],
[5]])

In [75]:
Oldx = A.inv()*b
Oldx

Matrix([
[-7],
[-4],
[-1]])

Let's see what we get now:

In [77]:
A = np.array([[1,-3,4],[-2,5,-7],[1,-5,8]])
print(A)

[[ 1 -3  4]
 [-2  5 -7]
 [ 1 -5  8]]


In [78]:
b = np.array([[1],[1],[5]])

**Note:** We can also directly transform a sy.Matrix into a np.array

A = np.array(A).astype(np.float64)

b = np.array(b).astype(np.float64)

Form the augemented matrix $M$:

In [51]:
M = np.hstack([A,b])
print(M)

[[ 1 -3  4  1]
 [-2  5 -7  1]
 [ 1 -5  8  5]]


Perform row operations:

In [52]:
M1 = add_row(M,2,1,0)
print(M1)

[[ 1. -3.  4.  1.]
 [ 0. -1.  1.  3.]
 [ 1. -5.  8.  5.]]


In [53]:
M2 = add_row(M1,-1,2,0)
print(M2)

[[ 1. -3.  4.  1.]
 [ 0. -1.  1.  3.]
 [ 0. -2.  4.  4.]]


In [54]:
M3 = scale_row(M2,-1,1)
print(M3)

[[ 1. -3.  4.  1.]
 [ 0.  1. -1. -3.]
 [ 0. -2.  4.  4.]]


In [55]:
M4 = add_row(M3,2,2,1)
print(M4)

[[ 1. -3.  4.  1.]
 [ 0.  1. -1. -3.]
 [ 0.  0.  2. -2.]]


In [56]:
M5 = scale_row(M4,1/2,2)
print(M5)

[[ 1. -3.  4.  1.]
 [ 0.  1. -1. -3.]
 [ 0.  0.  1. -1.]]


In [63]:
M6 = add_row(M5,1,1,2)
print(M6)

[[ 1. -3.  4.  1.]
 [ 0.  1.  0. -4.]
 [ 0.  0.  1. -1.]]


In [64]:
M7 = add_row(M6,-M6[0,2],0,2)
print(M7)

[[ 1. -3.  0.  5.]
 [ 0.  1.  0. -4.]
 [ 0.  0.  1. -1.]]


In [65]:
M8 = add_row(M7,-M7[0,1],0,1)
print(M8)

[[ 1.  0.  0. -7.]
 [ 0.  1.  0. -4.]
 [ 0.  0.  1. -1.]]


Success! The solution of $Ax=b$ is

In [67]:
x = M8[:,3].reshape(3,1)
print(x)

[[-7.]
 [-4.]
 [-1.]]


In [76]:
Oldx

Matrix([
[-7],
[-4],
[-1]])

Or, we can do it the easy way...

In [79]:
x = la.solve(A,b)
print(x)

[[-7.]
 [-4.]
 [-1.]]


## `scipy.linalg.solve`

We are mostly interested in linear systems $A \mathbf{x} = \mathbf{b}$ where there is a unique solution $\mathbf{x}$. This is the case when $A$ is a square matrix ($m=n$) and $\mathrm{det}(A) \not= 0$. To solve such a system, we can use the function [`scipy.linalg.solve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html).

The function returns a solution of the system of equations $A \mathbf{x} = \mathbf{b}$. For example:

In [50]:
A = np.array([[1,1],[1,-1]])
print(A)

[[ 1  1]
 [ 1 -1]]


In [51]:
b1 = np.array([2,0])
print(b1)

[2 0]


And solve:

In [52]:
x1 = la.solve(A,b1)
print(x1)

[1. 1.]


Note that the output $\mathbf{x}$ is returned as a 1D NumPy array when the vector $\mathbf{b}$ (the right hand side) is entered as a 1D NumPy array. If we input $\mathbf{b}$ as a 2D NumPy array, then the output is a 2D NumPy array. For example:

In [53]:
A = np.array([[1,1],[1,-1]])
b2 = np.array([2,0]).reshape(2,1)
x2 = la.solve(A,b2)
print(x2)

[[1.]
 [1.]]


Finally, if the right hand side $\mathbf{b}$ is a matrix, then the output is a matrix of the same size. It is the solution of $A \mathbf{x} = \mathbf{b}$ when $\mathbf{b}$ is a matrix. For example:

In [54]:
A = np.array([[1,1],[1,-1]])
b3 = np.array([[2,2],[0,1]])
x3 = la.solve(A,b3)
print(x3)

[[1.  1.5]
 [1.  0.5]]


### Simple Example

Let's compute the solution of the system of equations

\begin{align}
2x + y &= 1 \\\
x + y &= 1
\end{align}

Create the matrix of coefficients:

In [55]:
A = np.array([[2,1],[1,1]])
print(A)

[[2 1]
 [1 1]]


And the vector $\mathbf{b}$:

In [56]:
b = np.array([1,-1]).reshape(2,1)
print(b)

[[ 1]
 [-1]]


And solve:

In [57]:
x = la.solve(A,b)
print(x)

[[ 2.]
 [-3.]]


We can verify the solution by computing the inverse of $A$:

In [58]:
Ainv = la.inv(A)
print(Ainv)

[[ 1. -1.]
 [-1.  2.]]


And multiply $A^{-1} \mathbf{b}$ to solve for $\mathbf{x}$:

In [59]:
x = Ainv @ b
print(x)

[[ 2.]
 [-3.]]


We get the same result. Success!

### Inverse or Solve

It's a bad idea to use the inverse $A^{-1}$ to solve $A \mathbf{x} = \mathbf{b}$ if $A$ is large. It's too computationally expensive. Let's create a large random matrix $A$ and vector $\mathbf{b}$ and compute the solution $\mathbf{x}$ in 2 ways:

In [60]:
N = 1000
A = np.random.rand(N,N)
b = np.random.rand(N,1)

Check the first entries $A$:

In [61]:
A[:3,:3]

array([[0.35754719, 0.63135432, 0.6572258 ],
       [0.18450506, 0.14639832, 0.23528745],
       [0.27576474, 0.46264005, 0.26589724]])

And for $\mathbf{b}$:

In [62]:
b[:4,:]

array([[0.82726751],
       [0.96946096],
       [0.31351176],
       [0.63757837]])

Now we compare the speed of `scipy.linalg.solve` with `scipy.linalg.inv`:

In [65]:
%%timeit
x = la.solve(A,b)

2.77 s ± 509 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%%timeit
x = la.inv(A) @ b

4.46 s ± 2.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Solving with `scipy.linalg.solve` is about twice as fast!

## Exercises

Try the functions 'add_row', 'scale_row' and 'switch_row' for yourself and find the solution for the following system $Ax = b$ with 

$$
A = \begin{bmatrix}
6 & 15 & 1 \\
8 & 7 & 12 \\
2 & 7 & 8 
\end{bmatrix} \,, \quad 
b = \begin{bmatrix}
2 \\ 14 \\ 10 \\
\end{bmatrix}
$$